# Toolz

Toolz provides a set of utility functions for iterators, functions, and dictionaries. These functions interoperate well and form the building blocks of common data analytic operations. They extend the standard libraries itertools and functools and borrow heavily from the standard libraries of contemporary functional languages.

Toolz provides a suite of functions which have the following functional virtues:

- Composable: They interoperate due to their use of core data structures.
- Pure: They don’t change their inputs or rely on external state.
- Lazy: They don’t run until absolutely necessary, allowing them to support large streaming data sets.

Toolz functions are pragmatic. They understand that most programmers have deadlines.

- Low Tech: They’re just functions, no syntax or magic tricks to learn
- Tuned: They’re profiled and optimized
- Serializable: They support common solutions for parallel computing



## Best of

### Currying

In [1]:
from toolz import curry

def to_hex(num):
    return format(num, '02x')

def rgb(r, g, b):
    return f"#{to_hex(r)}{to_hex(g)}{to_hex(b)}"

print(rgb(255, 0, 0))
print(rgb(0, 255, 0))
print(rgb(0, 0, 255))

print("\n### Currying")
rgb = curry(rgb)
print(rgb(255))
print(rgb(255)(0))
print(rgb(255)(0)(0))

print("\n### blue")
blue = rgb(0)(0)
print(blue(255))
print(blue(128))

print("\n### red")
red = rgb(g=0, b=0)
print(red(255))
print(red(128))
print(red(64))

print("\n### greens")
green_to_blue = rgb(0)
print(green_to_blue(255, 128))
print(green_to_blue(128, 64))
print(green_to_blue(64, 255))



#ff0000
#00ff00
#0000ff

### Currying
<function rgb at 0x10638e7a0>
<function rgb at 0x10638e7a0>
#ff0000

### blue
#0000ff
#000080

### red
#ff0000
#800000
#400000

### greens
#00ff80
#008040
#0040ff


In [2]:
from IPython.display import HTML, display

# Display a colored square
def display_color(hex_color):
    display(HTML(f'<div style="background-color: {hex_color}; width: 50px; height: 50px; border: 1px solid black;"></div>'))

# Example usage
display_color(green_to_blue(255, 0))
display_color(green_to_blue(128, 128))
display_color(green_to_blue(64, 255))
display_color(red(0))
display_color(red(128))
display_color(red(255))

## Function Composition

Building complex functions by combining simpler ones

In [3]:
# Using toolz for composition
from toolz import compose
from operator import mul
from functools import partial

cube = lambda x: x ** 3
negate = lambda x: -x
process = compose(str, negate, cube)
print(process(3))

-27


In [4]:
# Create a text processing pipeline
process_text = compose(str.capitalize, " ".join, reversed, str.split, str.lower)

text = "The quick brown fox jumps over the lazy dog!"
print(process_text(text))

Dog! lazy the over jumps fox brown quick the


In [5]:
from toolz import compose_left

process_text = compose_left(str.lower, str.split, reversed, " ".join, str.capitalize)

text = "The quick brown fox jumps over the lazy dog!"
print(process_text(text))


Dog! lazy the over jumps fox brown quick the


In [6]:
from toolz import pipe

result = pipe(
    range(10),
    filter(lambda x: x % 2 == 0),
    map(lambda x: x / 2),
    sum
)

TypeError: filter expected 2 arguments, got 1

In [16]:
from toolz.curried import pipe, map, filter


result = pipe(
    range(10),
    filter(lambda x: x % 2 == 0), # [0, 2, 4, 6, 8]
    map(lambda x: x / 2),         # [0.0, 1.0, 2.0, 3.0, 4.0]
    sum,                          # 10.0
    str,                          # '10.0'
    reversed,                     # ['0', '.', '0', '1']
    "".join,                      # '0.01'
)
print(result)

0.01


## Dicttoolz

In [21]:
from toolz import valfilter

users = {
    'user1': {'name': 'Alice', 'age': 25, 'active': True},
    'user2': {'name': 'Bob', 'age': 17, 'active': False},
    'user3': {'name': 'Charlie', 'age': 32, 'active': True},
    'user4': {'name': 'Diana', 'age': 15, 'active': False}
}

active_users = valfilter(lambda user: user['active'], users)
print("Active users:")
for user in active_users.values():
    print(f"\t{user['name']}, {user['age']}")


Active users:
	Alice, 25
	Charlie, 32


In [ ]:
# More complex example
def remove_punctuation(text):
    import string
    return ''.join(c for c in text if c not in string.punctuation)

def lowercase(text):
    return text.lower()

def tokenize(text):
    return text.split()

def remove_stopwords(words):
    stopwords = {'the', 'a', 'an', 'in', 'on', 'at', 'for', 'to', 'of', 'with', 'by'}
    return [word for word in words if word not in stopwords]

# Create a text processing pipeline
process_text = compose(remove_stopwords, tokenize, lowercase, remove_punctuation)

text = "The quick brown fox jumps over the lazy dog!"
print(f"Original text: {text}")
print(f"Processed text: {process_text(text)}")

In [ ]:
from toolz.dicttoolz import get_in
import json

with open('users.json') as f:
    users = json.load(f)

for user in users:
    print(get_in(['address', 'geo', 'lat'], user))


-37.3159
-43.9509
-68.6102
29.4572
-31.8129
-71.4197
24.8918
-14.3990
24.6463
-38.2386
[-71.4197]


In [40]:
active_lat = pipe(
    users,
    filter(lambda user: get_in(['active'], user)),
    map(lambda user: get_in(['address', 'geo', 'lat'], user)),
    map(float),
    list
)

print(active_lat)

[-71.4197]


In [45]:
from toolz import groupby

fruits = [
    {'name': 'apple', 'color': 'red'},
    {'name': 'banana', 'color': 'yellow'},
    {'name': 'cherry', 'color': 'red'},
    {'name': 'grape', 'color': 'purple'},
    {'name': 'strawberry', 'color': 'red'},
    {'name': 'lemon', 'color': 'yellow'}
]

print("Grouping fruits by color:")
grouped_fruits = groupby('color', fruits)
for color, group in grouped_fruits.items():
    fruit_names = [fruit['name'] for fruit in group]
    print(f"{color}: {fruit_names}")

print(grouped_fruits)

Grouping fruits by color:
red: ['apple', 'cherry', 'strawberry']
yellow: ['banana', 'lemon']
purple: ['grape']
{'red': [{'name': 'apple', 'color': 'red'}, {'name': 'cherry', 'color': 'red'}, {'name': 'strawberry', 'color': 'red'}], 'yellow': [{'name': 'banana', 'color': 'yellow'}, {'name': 'lemon', 'color': 'yellow'}], 'purple': [{'name': 'grape', 'color': 'purple'}]}
